In [17]:
import enum
!pip install -qU langchain-openai
!pip install langchain_chroma
!pip install langchain_community
!pip install langchainhub
!pip install pypdf


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 399.1 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [31]:
import os
from api_key import OPENAI_API_KEY
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
llm = ChatOpenAI(model="gpt-4o-mini")

In [29]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
import enum

# Units class, to force the units to be of a certain value
class Units(str, enum.Enum):
    DOLLAR = "dollar"
    EURO = "euro"
    YEN = "yen"

# Pydantic class for formatting the LLM answer for one year
class RevenuePerYear(BaseModel):
    """Parsing information about revenue of the company for a given year."""
    company_name: str = Field(..., description="name of the company")
    year: int = Field(..., description="year of the revenue")
    revenue: float = Field(..., description="amount of revenue")
    unit: Units = Field(..., description="unit of revenue")

# Pydantic class that takes the revenue for all the years
class AnnualRevenues(BaseModel):
    """Parsing information about all available revenues of the company."""
    data: list[RevenuePerYear] = Field(..., description="Revenue for a given year", )

parser = PydanticOutputParser(pydantic_object=AnnualRevenues)

In [28]:
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# Load report
loader = PyPDFLoader(
    "annual-report-adidas-ar22.pdf",
)
docs = loader.load()

# Split the documents in chunks, create their embeddings and store them in a vector database
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Prompt that will be given to the model. It explains to use the retrieved chunks to answer the given question
prompt = PromptTemplate(
    template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. .\n{format_instructions}\nQuestion: {question} \nContext: {context} \nAnswer:",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create the retrieval and generation chain to answer the question using the relevant chunks of the report.
retriever = vectorstore.as_retriever()
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

# Call the chain : retrieval and answer generation will happen there
rag_chain.invoke("What adidas annual revenue per years?")

/home/pmalet/School/MIAGE/Memoire/LLM-Demo/venv/lib/python3.11/site-packages/pydantic/main.py:1328: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn(
/home/pmalet/School/MIAGE/Memoire/LLM-Demo/venv/lib/python3.11/site-packages/pydantic/_internal/_model_construction.py:268: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn(
/home/pmalet/School/MIAGE/Memoire/LLM-Demo/venv/lib/python3.11/site-packages/pydantic/_internal/_model_construction.py:268: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migr

AnnualRevenue(company_name='adidas AG', year=2022.0, revenue=4814.0, unit=<Units.EURO: 'euro'>)